
Importing Basic libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
sns.set()
tf.__version__

loading data

In [ ]:
styles = pd.read_csv('../input/fashion-product-images-small/styles.csv',nrows=4000, error_bad_lines = False)
styles['image'] = styles.apply(lambda row: str(row['id']) + ".jpg", axis=1)

In [ ]:
Apparel = styles[styles['masterCategory'].isin(['Apparel'])]
Accessories = styles[styles['masterCategory'].isin(['Accessories'])]
Footwear =  styles[styles['masterCategory'].isin(['Footwear'])]
Personal_Care =  styles[styles['masterCategory'].isin(['Personal Care'])]
Free_Items = styles[styles['masterCategory'].isin(['Free Items'])]
Sporting_Goods  =  styles[styles['masterCategory'].isin(['Sporting Goods'])]



Apparel, Accessories , Footwear, Personal_Care, Free_Items, Sporting_Goods= Apparel['id'].to_numpy(), Accessories['id'].to_numpy(), Footwear['id'].to_numpy(),Personal_Care['id'].to_numpy(), Free_Items['id'].to_numpy(), Sporting_Goods['id'].to_numpy()
Apparel.shape, Accessories.shape , Footwear.shape, Personal_Care.shape, Free_Items.shape, Sporting_Goods.shape

In [ ]:
image_path = '/kaggle/input/fashion-product-images-small/images/'

IMG_SIZE = 224
LIMIT_IMAGES = 2000
NUM_OUTPUTS = 3

In [ ]:
from tqdm import tqdm
import gc

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
def load_imgs(names):
    from tensorflow.keras.preprocessing import image
    imgs = []
    for i, image_name in enumerate(tqdm(names)):
        try:
            img = image.load_img(f'{image_path}{image_name}.jpg', target_size=(IMG_SIZE, IMG_SIZE))
            print(f'{image_path}{image_name}.jpg')
        except:
            img = None
            print("Error: Image not found!")
        if img is None:
            continue
        img = np.array(img)
        imgs.append(img)
    return np.array(imgs)

In [ ]:
print("Loading Images...")
print("Apparel")
Apparel_images = load_imgs(Apparel[:LIMIT_IMAGES])
gc.collect()
print("Accessories")
Accessories_images = load_imgs(Accessories[:LIMIT_IMAGES])
gc.collect()
print("Footwear")
Footwear_images = load_imgs(Footwear[:LIMIT_IMAGES])
gc.collect()
print("Done")
Apparel_images.shape, Accessories_images.shape, Footwear_images.shape

View example Image..:

In [ ]:
plt.imshow(Apparel_images[1])


Set up training and validation generators:

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

#image generator object from keras. reference : Keras Docs
image_generator = ImageDataGenerator(
    validation_split=0.2
)

#create a flow of images for training the model.
training_generator = image_generator.flow_from_dataframe(
    dataframe=styles,
    directory= "/kaggle/input/fashion-product-images-small/myntradataset/images",
    x_col="image",
    y_col="masterCategory",
    target_size=(80,60),
    batch_size=256,
    subset="training"

)

#create a flow of images for validating(testing) the trained model.
validation_generator = image_generator.flow_from_dataframe(
    dataframe=styles,
    directory="/kaggle/input/fashion-product-images-small/myntradataset/images",
    x_col="image",
    y_col="masterCategory",
    target_size=(80,60),
    batch_size=256,
    subset="validation"
)

Compile Convolutional Neural Network 

In [ ]:
from keras import layers,models
model1 = Sequential()
#model.add(Flatten(input_shape=(28, 28)))
model1.add(layers.Conv2D(32, (4,4), strides = (2,2), activation = 'relu' , input_shape = (80,60,3)))
model1.add(Flatten())
model1.add(Dense(units=128, activation='relu'))
model1.add(Dense(units=5, activation='softmax'))

model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model1.summary()



Train (CNN) Model:

In [ ]:
history = model1.fit(training_generator, epochs=10, validation_data=validation_generator, steps_per_epoch=10, verbose=1)

You can see metrics about model. 

In [ ]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy')
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

Small differences between accuracy and val_accuracy with accuracy >95% therefore the model looks relatively good.